# Capstone - Coffee shop

Import libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


## 1. Data - Preparation

#### Read list of addresses from url

In [2]:
df_address = pd.DataFrame()
df_address = pd.read_csv("https://raw.githubusercontent.com/phuonglehita/Coursera_Capstone/master/40Address.csv", delimiter = ',')
df_address.shape

(40, 2)

In [3]:
df = pd.DataFrame()
df = df_address

In [4]:
df['Latitude'] = ''
df['Longtitude'] = ''
df.head()

,Place,Address,Latitude,Longtitude
0,Position1,"132A Nguyễn Thái Học, Quận 1, Tp. Hồ Chí Minh,...",,
1,Position2,"180 Nguyễn Thị Minh Khai, Quận 3, Tp. Hồ Chí M...",,
2,Position3,"9 Mai Chi Thọ, Quận 2, Tp. Hồ Chí Minh, Việt Nam",,
3,Position4,"436 Nguyễn Thị Minh Khai, Quận 3, Tp. Hồ Chí M...",,
4,Position5,"46 Hoa Hồng, Quận Phú Nhuận, Tp. Hồ Chí Minh, ...",,


#### Get coordinates from addresses

In [5]:
for idx, row in df.iterrows():
#     print(row['Address'])
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(row['Address'])
    if (location != None):
        latitude = location.latitude
        longitude = location.longitude
        df.loc[idx,'Latitude'] = latitude
        df.loc[idx,'Longtitude'] = longitude

In [40]:
# df.to_csv('50AddressLatLong.csv', encoding='utf-16', sep='\t', index=False)

In [6]:
df.head()

,Place,Address,Latitude,Longtitude
0,Position1,"132A Nguyễn Thái Học, Quận 1, Tp. Hồ Chí Minh,...",10.7651,106.697
1,Position2,"180 Nguyễn Thị Minh Khai, Quận 3, Tp. Hồ Chí M...",10.7701,106.686
2,Position3,"9 Mai Chi Thọ, Quận 2, Tp. Hồ Chí Minh, Việt Nam",10.7809,106.738
3,Position4,"436 Nguyễn Thị Minh Khai, Quận 3, Tp. Hồ Chí M...",10.7701,106.686
4,Position5,"46 Hoa Hồng, Quận Phú Nhuận, Tp. Hồ Chí Minh, ...",10.7967,106.69


## 2. Using Foursquare for getting nearby venues

In [11]:
CLIENT_ID = 'F3QUDQPILOH1WYHOWYVKIVHL3CMKMKFVKEFQVWRYPN5RAS5A' # your Foursquare ID
CLIENT_SECRET = 'JYQVHQ10T0VNPKVTELEOQR2XW5ZXM1RO11QU1VZEZCCFIM11' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100
radius = 500
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: F3QUDQPILOH1WYHOWYVKIVHL3CMKMKFVKEFQVWRYPN5RAS5A
CLIENT_SECRET:JYQVHQ10T0VNPKVTELEOQR2XW5ZXM1RO11QU1VZEZCCFIM11


In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
             
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Place', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
position_venues = pd.DataFrame()
position_venues = getNearbyVenues(names=df['Place'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longtitude']
                                  )
position_venues.shape

Position1
Position2
Position3
Position4
Position5
Position6
Position7
Position8
Position9
Position10
Position11
Position12
Position13
Position14
Position15
Position16
Position17
Position18
Position19
Position20
Position21
Position22
Position23
Position24
Position25
Position26
Position27
Position28
Position29
Position30
Position31
Position32
Position33
Position34
Position35
Position36
Position37
Position38
Position39
Position40


(1019, 7)

In [18]:
position_venues.head()

,Place,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Position1,10.765127,106.697082,Ụt Ụt Restaurant,10.764427,106.698486,BBQ Joint
1,Position1,10.765127,106.697082,Quan Chay Chan Nhu,10.765242,106.694824,Vegetarian / Vegan Restaurant
2,Position1,10.765127,106.697082,Whiskey & Wares,10.766360,106.694433,Whisky Bar
3,Position1,10.765127,106.697082,Mekong Kitchen,10.767911,106.698050,Vietnamese Restaurant
4,Position1,10.765127,106.697082,Quince,10.765756,106.699680,Bistro


In [19]:
position_venues.shape

(1019, 7)

#### Number of categories

In [20]:
print('There are {} uniques categories.'.format(len(position_venues['Venue Category'].unique())))

There are 121 uniques categories.


In [21]:
position_venues['Venue Category'].unique()

array(['BBQ Joint', 'Vegetarian / Vegan Restaurant', 'Whisky Bar',
       'Vietnamese Restaurant', 'Bistro', 'Tattoo Parlor', 'Hostel',
       'Burger Joint', 'Mexican Restaurant', 'Massage Studio',
       'Juice Bar', 'Coffee Shop', 'Beer Bar', 'Ice Cream Shop', 'Hotel',
       'Café', 'Japanese Restaurant', 'Restaurant', 'Noodle House',
       'Cocktail Bar', 'Sushi Restaurant', 'Electronics Store',
       'Chinese Restaurant', 'Asian Restaurant',
       'Middle Eastern Restaurant', 'Food Truck', 'Lounge', 'Market',
       'Indian Restaurant', 'Bookstore', 'Bakery', 'Sandwich Place',
       'Snack Place', 'Movie Theater', 'Bubble Tea Shop', 'Soup Place',
       'Fast Food Restaurant', 'Seafood Restaurant', 'Music Venue',
       'Dessert Shop', 'Italian Restaurant', 'Supermarket', 'Pizza Place',
       'French Restaurant', 'Spa', 'Yoga Studio', 'Design Studio',
       'Furniture / Home Store', 'Stadium', 'Pool', 'Thai Restaurant',
       'Diner', 'Shopping Plaza', 'Breakfast Spot',
  

## 3. Analyze Each Position

In [22]:
position_fre = pd.DataFrame()
position_fre = pd.get_dummies(position_venues[['Venue Category']], prefix="", prefix_sep="")

# add Place column back to dataframe
position_fre['Place'] = position_venues['Place'] 

# move Place column to the first column
fixed_columns = [position_fre.columns[-1]] + list(position_fre.columns[:-1])
position_fre = position_fre[fixed_columns]

position_fre.head()


,Place,American Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Bistro,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Café,Cantonese Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,Convenience Store,Cupcake Shop,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food Court,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Furniture / Home Store,German Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Lounge,Market,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Venue,Nightclub,Noodle House,North Indian Restaurant,Opera House,Park,Pet Café,Pizza Place,Pool,Pub,Ramen Restaurant,Residential Building (Apartment / Condo),Resort,Restaurant,Road,Sandwich Place,Scandinavian Restaurant,Seafood Restaurant,Shopping Mall,Shopping Plaza,Snack Place,Soup Place,Spa,Spanish Restaurant,Speakeasy,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Thai Restaurant,Theater,Travel Agency,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Yoga Studio
0,Position1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Position1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,Position1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,Position1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,Position1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### Take the mean of the frequency of occurrence of each category

In [23]:
position_grouped = position_fre.groupby('Place').mean().reset_index()
position_grouped

,Place,American Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Bistro,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Café,Cantonese Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,Convenience Store,Cupcake Shop,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food Court,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Furniture / Home Store,German Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Lounge,Market,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Venue,Nightclub,Noodle House,North Indian Restaurant,Opera House,Park,Pet Café,Pizza Place,Pool,Pub,Ramen Restaurant,Residential Building (Apartment / Condo),Resort,Restaurant,Road,Sandwich Place,Scandinavian Restaurant,Seafood Restaurant,Shopping Mall,Shopping Plaza,Snack Place,Soup Place,Spa,Spanish Restaurant,Speakeasy,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Thai Restaurant,Theater,Travel Agency,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Yoga Studio
0,Position1,0.000000,0.000000,0.000000,0.000000,0.028571,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.028571,0.000000,0.028571,0.000000,0.000000,0.000000,0.000000,0.000000,0.028571,0.028571,0.000000,0.000000,0.000000,0.000000,0.085714,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.057143,0.114286,0.000000,0.000000,0.028571,0.000000,0.000000,0.028571,0.000000,0.000000,0.057143,0.000000,0.000000,0.0,0.000000,0.000000,0.028571,0.000000,0.028571,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.00,0.000000,0.000000,0.028571,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.028571,0.000000,0.000000,0.000000,0.000000,0.057143,0.000000,0.285714,0.028571,0.000000
1,Position10,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.125,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.125,0.00,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Position11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,

#### Display top 5 common venue category of each position

In [25]:
num_top_venues = 5

for pos in position_grouped['Place']:
    print("----"+pos+"----")
    temp = position_grouped[position_grouped['Place'] == pos].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Position1----
                   venue  freq
0  Vietnamese Restaurant  0.29
1                  Hotel  0.11
2            Coffee Shop  0.09
3              Juice Bar  0.06
4                 Hostel  0.06


----Position10----
                venue  freq
0  Seafood Restaurant  0.25
1      Breakfast Spot  0.12
2              Bakery  0.12
3    Football Stadium  0.12
4       Movie Theater  0.12


----Position11----
                   venue  freq
0                   Café  0.75
1  Vietnamese Restaurant  0.25
2    American Restaurant  0.00
3      Mobile Phone Shop  0.00
4            Pizza Place  0.00


----Position12----
                   venue  freq
0              BBQ Joint  0.50
1  Vietnamese Restaurant  0.25
2                   Café  0.25
3    American Restaurant  0.00
4           Noodle House  0.00


----Position13----
                 venue  freq
0    Mobile Phone Shop  0.17
1     Asian Restaurant  0.17
2  Japanese Restaurant  0.17
3           Bagel Shop  0.17
4       Baseball Field  0.1

#### Sort the venues in descending order

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [27]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Place']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Place'] = position_grouped['Place']

for ind in np.arange(position_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(position_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Place,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Position1,Vietnamese Restaurant,Hotel,Coffee Shop,Hostel,Vegetarian / Vegan Restaurant,Juice Bar,Ice Cream Shop,Japanese Restaurant,Restaurant,Massage Studio
1,Position10,Seafood Restaurant,Breakfast Spot,Movie Theater,Pub,Football Stadium,Bakery,Restaurant,Flea Market,Fast Food Restaurant,Electronics Store
2,Position11,Café,Vietnamese Restaurant,Yoga Studio,Food Truck,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dumpling Restaurant,Eastern European Restaurant
3,Position12,BBQ Joint,Vietnamese Restaurant,Café,Yoga Studio,Electronics Store,Food,Flower Shop,Flea Market,Fast Food Restaurant,Dumpling Restaurant
4,Position13,Baseball Field,Mobile Phone Shop,Asian Restaurant,Japanese Restaurant,Bagel Shop,Fast Food Restaurant,Electronics Store,Food,Flower Shop,Flea Market


## 4. Cluster Neighborhoods

#### Run *k*-means to cluster the neighborhood into 5 clusters.

In [66]:
# set number of clusters
kclusters = 5

position_grouped_clustering = position_grouped.drop('Place', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(position_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 4, 1, 0, 4, 0, 0, 1, 1, 4], dtype=int32)

In [98]:
# add clustering labels

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

df_merged = df
df_merged = df_merged.join(neighborhoods_venues_sorted.set_index('Place'), on='Place')
df_merged = df_merged.fillna(0)
df_merged['Cluster Labels'] = df_merged['Cluster Labels'].astype(int)
df_merged['Cluster Labels']
df_merged.head() # check the last columns!

,Place,Address,Latitude,Longtitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Position1,"132A Nguyễn Thái Học, Quận 1, Tp. Hồ Chí Minh,...",10.765127,106.697082,0,Vietnamese Restaurant,Hotel,Coffee Shop,Hostel,Vegetarian / Vegan Restaurant,Juice Bar,Ice Cream Shop,Japanese Restaurant,Restaurant,Massage Studio
1,Position2,"180 Nguyễn Thị Minh Khai, Quận 3, Tp. Hồ Chí M...",10.770121,106.686220,0,Vietnamese Restaurant,Café,Bookstore,Hotel,Chinese Restaurant,Bakery,Vegetarian / Vegan Restaurant,Asian Restaurant,Sushi Restaurant,Dessert Shop
2,Position3,"9 Mai Chi Thọ, Quận 2, Tp. Hồ Chí Minh, Việt Nam",10.780942,106.737842,1,Café,Bubble Tea Shop,Yoga Studio,Food Court,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dumpling Restaurant,Eastern European Restaurant
3,Position4,"436 Nguyễn Thị Minh Khai, Quận 3, Tp. Hồ Chí M...",10.770121,106.686220,0,Vietnamese Restaurant,Café,Bookstore,Hotel,Chinese Restaurant,Bakery,Vegetarian / Vegan Restaurant,Asian Restaurant,Sushi Restaurant,Dessert Shop
4,Position5,"46 Hoa Hồng, Quận Phú Nhuận, Tp. Hồ Chí Minh, ...",10.796693,106.689885,4,Coffee Shop,Café,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Yoga Studio,Japanese Restaurant,Design Studio,Diner,Pool,Juice Bar


#### Get coordinates of Ho Chi Minh City

In [99]:
# Get coordinates of District 1 - Center of Ho Chi Minh City
address = 'District 1, Ho Chi Minh city, Viet Nam'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [101]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster, add in zip(df_merged['Latitude'], df_merged['Longtitude'], df_merged['Place'], df_merged['Cluster Labels'], df_merged['Address']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster) + ' - ' + add , parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Cluster 1

In [82]:
df_merged.loc[df_merged['Cluster Labels'] == 0, df_merged.columns[[1] + list(range(5, df_merged.shape[1]))]]

,Address,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,"399 Phan Huy Ích, Quận Gò Vấp, Tp. Hồ Chí Minh...",Mobile Phone Shop,Japanese Restaurant,Bagel Shop,Fast Food Restaurant,Flea Market,Football Stadium,Food Truck,Food Court,Food,Flower Shop


#### Cluster 2

In [83]:
df_merged.loc[df_merged['Cluster Labels'] == 1, df_merged.columns[[1] + list(range(5, df_merged.shape[1]))]]

,Address,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"132A Nguyễn Thái Học, Quận 1, Tp. Hồ Chí Minh,...",Vietnamese Restaurant,Hotel,Japanese Restaurant,Hostel,Vegetarian / Vegan Restaurant,Coffee Shop,Juice Bar,Beer Bar,Café,Burger Joint
1,"180 Nguyễn Thị Minh Khai, Quận 3, Tp. Hồ Chí M...",Vietnamese Restaurant,Café,Bookstore,Hotel,Chinese Restaurant,Vegetarian / Vegan Restaurant,Asian Restaurant,Bakery,Sushi Restaurant,Dessert Shop
2,"9 Mai Chi Thọ, Quận 2, Tp. Hồ Chí Minh, Việt Nam",Bakery,Café,Bubble Tea Shop,Yoga Studio,French Restaurant,Dive Bar,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Fast Food Restaurant
3,"436 Nguyễn Thị Minh Khai, Quận 3, Tp. Hồ Chí M...",Vietnamese Restaurant,Café,Bookstore,Hotel,Chinese Restaurant,Vegetarian / Vegan Restaurant,Asian Restaurant,Bakery,Sushi Restaurant,Dessert Shop
4,"46 Hoa Hồng, Quận Phú Nhuận, Tp. Hồ Chí Minh, ...",Coffee Shop,Café,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Stadium,Japanese Restaurant,Furniture / Home Store,Juice Bar,Diner,Design Studio
5,"154 Nguyễn Xí, Quận Bình Thạnh, Tp. Hồ Chí Min...",Vietnamese Restaurant,Café,Breakfast Spot,Bookstore,Convenience Store,Food Truck,Snack Place,Dessert Shop,Asian Restaurant,Gym / Fitness Center
7,"359A Lê Quang Đinh, Quận Bình Thạnh, Tp. Hồ Ch...",Vietnamese Restaurant,Asian Restaurant,Train Station,Café,Yoga Studio,Flea Market,Food Truck,Food Court,Food,Flower Shop
8,"2B Lê Duẩn, Quận 1, Tp. Hồ Chí Minh, Việt Nam",Coffee Shop,Café,Vietnamese Restaurant,Hotel,Asian Restaurant,Lounge,Hotel Bar,Italian Restaurant,Fast Food Restaurant,Restaurant
11,"310 Cây Trâm, Quận Gò Vấp, Tp. Hồ Chí Minh, Vi...",BBQ Joint,Vietnamese Restaurant,Café,Yoga Studio,Flea Market,Food Truck,Food Court,Food,Flower Shop,Electronics Store
13,"45 Lý Chính Thắng , Quận 3 , Tp. Hồ Chí Minh, ...",Café,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Asian Restaurant,Coffee Shop,Seafood Restaurant,French Restaurant,Yoga Studio,Bar,Bookstore


#### Cluster 3

In [84]:
df_merged.loc[df_merged['Cluster Labels'] == 2, df_merged.columns[[1] + list(range(5, df_merged.shape[1]))]]

,Address,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,"68/5 Quang Trung, Quận Gò Vấp, Tp. Hồ Chí Minh...",Café,Vietnamese Restaurant,Yoga Studio,Football Stadium,Dim Sum Restaurant,Diner,Dive Bar,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
15,"23A Nguyễn Ảnh Thủ, Quận 12, Tp. Hồ Chí Minh,...",Café,Vietnamese Restaurant,Flower Shop,Yoga Studio,Football Stadium,Dim Sum Restaurant,Diner,Dive Bar,Dumpling Restaurant,Eastern European Restaurant
16,"371 Nguyễn Ảnh Thủ, Quận 12, Tp. Hồ Chí Minh, ...",Café,Vietnamese Restaurant,Flower Shop,Yoga Studio,Football Stadium,Dim Sum Restaurant,Diner,Dive Bar,Dumpling Restaurant,Eastern European Restaurant
18,"26 Ung Văn Khiêm, Quận Bình Thạnh, Tp. Hồ Chí ...",Café,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Yoga Studio,Fast Food Restaurant,Food Court,Food,Flower Shop,Flea Market,Eastern European Restaurant
24,"721 Huỳnh Tấn Phát, Quận 7, Tp. Hồ Chí Minh, V...",Pizza Place,Café,Yoga Studio,Football Stadium,Diner,Dive Bar,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Fast Food Restaurant


#### Cluster 4

In [85]:
df_merged.loc[df_merged['Cluster Labels'] == 3, df_merged.columns[[1] + list(range(5, df_merged.shape[1]))]]

,Address,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,"68/5 Quang Trung, Quận Gò Vấp, Tp. Hồ Chí Minh...",Café,Vietnamese Restaurant,Yoga Studio,Football Stadium,Dim Sum Restaurant,Diner,Dive Bar,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
15,"23A Nguyễn Ảnh Thủ, Quận 12, Tp. Hồ Chí Minh,...",Café,Vietnamese Restaurant,Flower Shop,Yoga Studio,Football Stadium,Dim Sum Restaurant,Diner,Dive Bar,Dumpling Restaurant,Eastern European Restaurant
16,"371 Nguyễn Ảnh Thủ, Quận 12, Tp. Hồ Chí Minh, ...",Café,Vietnamese Restaurant,Flower Shop,Yoga Studio,Football Stadium,Dim Sum Restaurant,Diner,Dive Bar,Dumpling Restaurant,Eastern European Restaurant
18,"26 Ung Văn Khiêm, Quận Bình Thạnh, Tp. Hồ Chí ...",Café,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Yoga Studio,Fast Food Restaurant,Food Court,Food,Flower Shop,Flea Market,Eastern European Restaurant
24,"721 Huỳnh Tấn Phát, Quận 7, Tp. Hồ Chí Minh, V...",Pizza Place,Café,Yoga Studio,Football Stadium,Diner,Dive Bar,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Fast Food Restaurant


#### Cluster 5

In [86]:
df_merged.loc[df_merged['Cluster Labels'] == 4, df_merged.columns[[1] + list(range(5, df_merged.shape[1]))]]

,Address,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,"287 Âu Cơ,Quận Tân Phú, Tp. Hồ Chí Minh, Việt Nam",Restaurant,Café,Shopping Plaza,Movie Theater,Yoga Studio,Food Court,Dim Sum Restaurant,Diner,Dive Bar,Dumpling Restaurant
9,"117 Nguyễn Văn Quá, Quận 12, Tp. Hồ Chí Minh, ...",Pub,Restaurant,Football Stadium,Café,Movie Theater,Diner,Dive Bar,Dumpling Restaurant,Food Court,Electronics Store
21,"57 Lê Đức Thọ, Quận Gò Vấp, Tp. Hồ Chí Minh, ...",Restaurant,Vietnamese Restaurant,Café,Lake,Yoga Studio,Fast Food Restaurant,Food,Flower Shop,Flea Market,Eastern European Restaurant
